# Artifacts for Pattern Recognition Project
## Authors: Ekansh Bhatnagar, Meghana Chintalapati

Install required packages mentioned in requirements.txt
absl-py==2.1.0
aiohappyeyeballs==2.4.3
aiohttp==3.11.8
aiosignal==1.3.1
anyio==4.6.2.post1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
astunparse==1.6.3
async-lru==2.0.4
async-timeout==5.0.1
attrs==24.2.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.2.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.4.0
colorama==0.4.6
comm==0.2.2
contourpy==1.3.1
cycler==0.12.1
debugpy==1.8.9
decorator==5.1.1
defusedxml==0.7.1
emnist==0.0
exceptiongroup==1.2.2
executing==2.1.0
fastjsonschema==2.21.0
filelock==3.16.1
flatbuffers==24.3.25
fonttools==4.55.0
fqdn==1.5.1
frozenlist==1.5.0
fsspec==2024.10.0
gast==0.6.0
google-pasta==0.2.0
grpcio==1.68.0
h11==0.14.0
h5py==3.12.1
httpcore==1.0.7
httpx==0.28.0
idna==3.10
ipykernel==6.29.5
ipython==8.30.0
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.4
joblib==1.4.2
json5==0.10.0
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyter_core==5.7.2
jupyter_server==2.14.2
jupyter_server_terminals==0.5.3
jupyterlab==4.3.1
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
keras==3.7.0
kiwisolver==1.4.7
libclang==18.1.1
lightning-utilities==0.11.9
Markdown==3.7
markdown-it-py==3.0.0
MarkupSafe==3.0.2
matplotlib==3.9.2
matplotlib-inline==0.1.7
mdurl==0.1.2
mistune==3.0.2
ml-dtypes==0.4.1
mpmath==1.3.0
multidict==6.1.0
namex==0.0.8
nbclient==0.10.1
nbconvert==7.16.4
nbformat==5.10.4
nest-asyncio==1.6.0
networkx==3.4.2
notebook_shim==0.2.4
numpy==2.0.2
opt_einsum==3.4.0
optree==0.13.1
overrides==7.7.0
packaging==24.2
pandas==2.2.3
pandocfilters==1.5.1
parso==0.8.4
pillow==11.0.0
platformdirs==4.3.6
prometheus_client==0.21.0
prompt_toolkit==3.0.48
propcache==0.2.0
protobuf==5.29.0
psutil==6.1.0
pure_eval==0.2.3
pycparser==2.22
Pygments==2.18.0
pyparsing==3.2.0
python-dateutil==2.9.0.post0
python-json-logger==2.0.7
python-version==0.0.2
pytorch-lightning==2.4.0
pytz==2024.2
pywin32==308
pywinpty==2.0.14
PyYAML==6.0.2
pyzmq==26.2.0
referencing==0.35.1
requests==2.32.3
rfc3339-validator==0.1.4
rfc3986-validator==0.1.1
rich==13.9.4
rpds-py==0.21.0
scikit-learn==1.5.2
scipy==1.14.1
Send2Trash==1.8.3
six==1.16.0
sniffio==1.3.1
soupsieve==2.6
stack-data==0.6.3
sympy==1.13.1
tensorboard==2.18.0
tensorboard-data-server==0.7.2
tensorflow==2.18.0
tensorflow-io-gcs-filesystem==0.31.0
tensorflow_intel==2.18.0
termcolor==2.5.0
terminado==0.18.1
threadpoolctl==3.5.0
tinycss2==1.4.0
tomli==2.2.1
torch==2.5.1
torchmetrics==1.6.0
torchvision==0.20.1
tornado==6.4.2
tqdm==4.67.1
traitlets==5.14.3
types-python-dateutil==2.9.0.20241003
typing_extensions==4.12.2
tzdata==2024.2
uri-template==1.3.0
urllib3==2.2.3
wcwidth==0.2.13
webcolors==24.11.1
webencodings==0.5.1
websocket-client==1.8.0
Werkzeug==3.1.3
wrapt==1.17.0
yarl==1.18.0

## Classes:

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics import accuracy_score
from scipy import ndimage
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise_distances
import pytorch_lightning as pl
import torch
from torchvision.models import resnet18
from torch import nn


"""This script defines the training, validation and testing process.
"""


# SVM with polynomial kernel
class SVM(object):
    def __init__(self, degree):
        self.degree = degree

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel='poly', degree=self.degree, coef0=1.0)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with translational-invariant kernel
class TISVM(object):
    def __init__(self, degree):
        self.degree = degree

    def ti_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for k in range(-5, 6):
            for l in range(-5, 6):
                # shift
                T_kl = np.roll(y_reshape, (k, l), axis=(1, 2))
                T_kl_reshape = T_kl.reshape((T_kl.shape[0], -1))

                kernel_medium = polynomial_kernel(x, T_kl_reshape, self.degree)
                kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ti_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with translational-invariant kernel
class TIRISVM(object):
    def __init__(self, degree):
        self.degree = degree

    def ti_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for k in range(-7, 8):
            for l in range(-7, 8):
                for d in range(-10, 10):
                    # shift
                    T_kl = np.roll(y_reshape, (k, l), axis=(1, 2))
                    # rotate
                    T_rotate = ndimage.rotate(T_kl, d, axes=(1, 2), reshape=False)
                    T_kl_reshape = T_rotate.reshape((T_kl.shape[0], -1))

                    kernel_medium = polynomial_kernel(x, T_kl_reshape, self.degree)
                    kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ti_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with translational-invariant kernel
class LTIRISVM(object):
    def __init__(self, degree, filter):
        self.degree = degree
        self.filter = filter

    def ti_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for k in range(-7, 8):
            for l in range(-7, 8):
                for d in range(-5, 6):
                    # rotate
                    T_rotate = ndimage.rotate(y_reshape, d, axes=(1, 2), reshape=False)
                    # shift
                    T_kl = np.roll(T_rotate, (k, l), axis=(1, 2))
                    T_kl_reshape = T_kl.reshape((T_kl.shape[0], -1))

                    kernel_medium = self.locality_kernel(x, T_kl_reshape)
                    kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    # using im2col to accelerate computation (memory locality)
    def im2col(self, x, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = x.shape

        assert (H + 2 * pad - filter_h) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Height'
        assert (W + 2 * pad - filter_w) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Width'
        out_h = (H + 2 * pad - filter_h) // stride + 1
        out_w = (W + 2 * pad - filter_w) // stride + 1

        img = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
        col = np.zeros((N, C, out_h, out_w, filter_h, filter_w))

        for y in range(out_h):
            for x in range(out_w):
                col[:, :, y, x, :, :] = img[:, :, y * stride:y * stride + filter_h, x * stride:x * stride + filter_w]

        col = col.transpose(0, 2, 3, 4, 5, 1).reshape((N, -1))
        return col

    def locality_kernel(self, x, y):
        X1 = np.transpose(x.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))
        Y1 = np.transpose(y.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))

        Z = np.zeros((x.shape[0], y.shape[0]))

        # using im2col to accelerate computation (memory locality)
        filter_h = self.filter
        filter_w = self.filter
        stride = 1
        pad = (self.filter - 1) // 2
        X2 = self.im2col(X1, filter_h, filter_w, stride, pad)
        Y2 = self.im2col(Y1, filter_h, filter_w, stride, pad)

        # Hyperparameter: Degree
        D1 = 2
        D2 = self.degree // D1

        for i in range(0, X2.shape[1], filter_h * filter_w):
            Z = Z + polynomial_kernel(X2[:, i: i + filter_h * filter_w], Y2[:, i: i + filter_h * filter_w],
                                      degree=D1, coef0=1.0)

        Z = (1 / (X1.shape[1] * X1.shape[2]) * Z + np.ones((x.shape[0], y.shape[0]))) ** D2
        return Z

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ti_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with translational-invariant kernel
class LTISVM(object):
    def __init__(self, degree, filter):
        self.degree = degree
        self.filter = filter

    def ti_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for k in range(-7, 8):
            for l in range(-7, 8):
                # shift
                T_kl = np.roll(y_reshape, (k, l), axis=(1, 2))
                T_kl_reshape = T_kl.reshape((T_kl.shape[0], -1))

                kernel_medium = self.locality_kernel(x, T_kl_reshape)
                kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    # using im2col to accelerate computation (memory locality)
    def im2col(self, x, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = x.shape

        assert (H + 2 * pad - filter_h) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Height'
        assert (W + 2 * pad - filter_w) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Width'
        out_h = (H + 2 * pad - filter_h) // stride + 1
        out_w = (W + 2 * pad - filter_w) // stride + 1

        img = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
        col = np.zeros((N, C, out_h, out_w, filter_h, filter_w))

        for y in range(out_h):
            for x in range(out_w):
                col[:, :, y, x, :, :] = img[:, :, y * stride:y * stride + filter_h, x * stride:x * stride + filter_w]

        col = col.transpose(0, 2, 3, 4, 5, 1).reshape((N, -1))
        return col

    def locality_kernel(self, x, y):
        X1 = np.transpose(x.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))
        Y1 = np.transpose(y.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))

        Z = np.zeros((x.shape[0], y.shape[0]))

        # using im2col to accelerate computation (memory locality)
        filter_h = self.filter
        filter_w = self.filter
        stride = 1
        pad = (self.filter - 1) // 2
        X2 = self.im2col(X1, filter_h, filter_w, stride, pad)
        Y2 = self.im2col(Y1, filter_h, filter_w, stride, pad)

        # Hyperparameter: Degree
        D1 = 2
        D2 = self.degree // D1

        for i in range(0, X2.shape[1], filter_h * filter_w):
            Z = Z + polynomial_kernel(X2[:, i: i + filter_h * filter_w], Y2[:, i: i + filter_h * filter_w],
                                      degree=D1, coef0=1.0)

        Z = (1 / (X1.shape[1] * X1.shape[2]) * Z + np.ones((x.shape[0], y.shape[0]))) ** D2
        return Z

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ti_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with local connective kernel
class LOCSVM(object):
    def __init__(self, degree, filter):
        self.degree = degree
        self.filter = filter

    # using im2col to accelerate computation (memory locality)
    def im2col(self, x, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = x.shape

        assert (H + 2 * pad - filter_h) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Height'
        assert (W + 2 * pad - filter_w) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Width'
        out_h = (H + 2 * pad - filter_h) // stride + 1
        out_w = (W + 2 * pad - filter_w) // stride + 1

        img = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
        col = np.zeros((N, C, out_h, out_w, filter_h, filter_w))

        for y in range(out_h):
            for x in range(out_w):
                col[:, :, y, x, :, :] = img[:, :, y * stride:y * stride + filter_h, x * stride:x * stride + filter_w]

        col = col.transpose(0, 2, 3, 4, 5, 1).reshape((N, -1))
        return col

    def locality_kernel(self, x, y):
        X1 = np.transpose(x.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))
        Y1 = np.transpose(y.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))

        Z = np.zeros((x.shape[0], y.shape[0]))

        # using im2col to accelerate computation (memory locality)
        filter_h = self.filter
        filter_w = self.filter
        stride = 1
        pad = (self.filter - 1) // 2
        X2 = self.im2col(X1, filter_h, filter_w, stride, pad)
        Y2 = self.im2col(Y1, filter_h, filter_w, stride, pad)

        # Hyperparameter: Degree
        D1 = 2
        D2 = self.degree // D1

        for i in range(0, X2.shape[1], filter_h * filter_w):
            Z = Z + polynomial_kernel(X2[:, i: i + filter_h * filter_w], Y2[:, i: i + filter_h * filter_w],
                                      degree=D1, coef0=1.0)

        Z = (1 / (X1.shape[1] * X1.shape[2]) * Z + np.ones((x.shape[0], y.shape[0]))) ** D2
        return Z

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.locality_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with rotation-invariant kernel
class RISVM(object):
    def __init__(self, degree):
        self.degree = degree

    def ri_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for d in range(-10, 10):
            # rotate
            T_rotate = ndimage.rotate(y_reshape, d, axes=(1, 2), reshape=False)
            T_rotate_reshape = T_rotate.reshape((T_rotate.shape[0], -1))

            kernel_medium = polynomial_kernel(x, T_rotate_reshape, self.degree)
            kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ri_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with rotation-invariant kernel
class RIISVM(object):
    def __init__(self, degree):
        self.degree = degree

    def ri_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for d in range(-10, 10):
            # rotate
            T_rotate = ndimage.rotate(y_reshape, d, axes=(1, 2), reshape=False)
            T_rotate_reshape = T_rotate.reshape((T_rotate.shape[0], -1))

            kernel_final = kernel_final + polynomial_kernel(x, T_rotate_reshape, self.degree)

        return kernel_final/20

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ri_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with rotation-invariant kernel
class LRISVM(object):
    def __init__(self, degree, filter):
        self.degree = degree
        self.filter = filter

    def ri_kernel(self, x, y):
        """
        Args:
            x: arrays of shape (n_samples1, n_features)
            y: arrays of shape (n_samples2, n_features)
        Returns:
            kernel_final: maximum kernel matrix of shape (n_samples1, n_samples2)
        """
        y_reshape = y.reshape((-1, 28, 28))

        # choose the maximum kernel for each pair of sample
        kernel_final = np.zeros((x.shape[0], y.shape[0]))

        for d in range(-5, 6):
            # rotate
            T_rotate = ndimage.rotate(y_reshape, d, axes=(1, 2), reshape=False)
            T_rotate_reshape = T_rotate.reshape((T_rotate.shape[0], -1))

            kernel_medium = self.locality_kernel(x, T_rotate_reshape)
            kernel_final = np.maximum(kernel_medium, kernel_final)

        return kernel_final

    # using im2col to accelerate computation (memory locality)
    def im2col(self, x, filter_h, filter_w, stride=1, pad=0):
        N, C, H, W = x.shape

        assert (H + 2 * pad - filter_h) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Height'
        assert (W + 2 * pad - filter_w) % stride == 0, 'Sanity Check Status: Conv Layer Failed in Width'
        out_h = (H + 2 * pad - filter_h) // stride + 1
        out_w = (W + 2 * pad - filter_w) // stride + 1

        img = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
        col = np.zeros((N, C, out_h, out_w, filter_h, filter_w))

        for y in range(out_h):
            for x in range(out_w):
                col[:, :, y, x, :, :] = img[:, :, y * stride:y * stride + filter_h, x * stride:x * stride + filter_w]

        col = col.transpose(0, 2, 3, 4, 5, 1).reshape((N, -1))
        return col

    def locality_kernel(self, x, y):
        X1 = np.transpose(x.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))
        Y1 = np.transpose(y.reshape((-1, 28, 28, 1)), (0, 3, 1, 2))

        Z = np.zeros((x.shape[0], y.shape[0]))

        # using im2col to accelerate computation (memory locality)
        filter_h = self.filter
        filter_w = self.filter
        stride = 1
        pad = (self.filter - 1) // 2
        X2 = self.im2col(X1, filter_h, filter_w, stride, pad)
        Y2 = self.im2col(Y1, filter_h, filter_w, stride, pad)

        # Hyperparameter: Degree
        D1 = 2
        D2 = self.degree // D1

        for i in range(0, X2.shape[1], filter_h * filter_w):
            Z = Z + polynomial_kernel(X2[:, i: i + filter_h * filter_w], Y2[:, i: i + filter_h * filter_w],
                                      degree=D1, coef0=1.0)

        Z = (1 / (X1.shape[1] * X1.shape[2]) * Z + np.ones((x.shape[0], y.shape[0]))) ** D2

        return Z

    def train(self, x, y):
        svclassifier = SVC(gamma='auto', kernel=self.ri_kernel)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# kNN with tangent distance
class KNN(object):
    def __init__(self, metrics):
        self.metrics = metrics

    def train(self, x, y):
        neigh = KNeighborsClassifier(n_neighbors=3, metric=self.metrics)
        neigh.fit(x, y)
        y_pred = neigh.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return neigh, train_acc

    def evaluate(self, x, y, neigh):
        y_pred = neigh.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# SVM with tangent distance
class TDSVM(object):
    def __init__(self, metrics):
        self.metrics = metrics

    def TD(self, x, y):
        return np.exp(-pairwise_distances(x, y, metric=self.metrics)**2 / 784)

    def train(self, x, y):
        svclassifier = SVC(gamma=1, kernel=self.TD)
        svclassifier.fit(x, y)
        y_pred = svclassifier.predict(x)
        train_acc = accuracy_score(y, y_pred)
        return svclassifier, train_acc

    def evaluate(self, x, y, svclassifier):
        y_pred = svclassifier.predict(x)
        eval_acc = accuracy_score(y, y_pred)
        return eval_acc


# ResNet
class ResNetModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_no):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.005)

# To run for TB dataset run the code in next cell. 

In [ ]:
import numpy as np
import time
from Model import SVM, TISVM, RISVM, LOCSVM, LTISVM, LRISVM, TIRISVM, LTIRISVM, RIISVM, KNN, TDSVM, ResNetModel
from DataLoader import load_data, train_valid_split
import ctypes
import torch
import pytorch_lightning as pl
import torch.utils.data as data_utils
from tqdm.autonotebook import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# Preprocess images
def preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        full_path = os.path.join(image_dir, img_path)
        img = load_img(full_path, target_size=(224, 224))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)
    
def preprocess_images_svm(image_paths):
    images = []
    for img_path in image_paths:
        full_path = os.path.join(image_dir, img_path)
        img = load_img(full_path, target_size=(28, 28),color_mode='grayscale')  # Resize for simplicity
        img_array = img_to_array(img)
        img_array = img_array.flatten() / 255.0  # Flatten and normalize
        images.append(img_array)
    return np.array(images)

# Load the CSV file
file_path = 'TB/tb_metadata.csv'
df = pd.read_csv(file_path)

samples_num = [100, 200, 400]
results = open("result.txt","w",buffering=1)
results.write("model,number_of_samples,test_accuracy\n")

for num in samples_num:
    # Select the first 100 entries
    subset_df = df[['PATH', 'Label']].head(num)

    # Get the lists of image paths and labels from the sample
    sample_image_paths = subset_df['PATH'].tolist()
    sample_labels = subset_df['Label'].tolist()

    test_set_df = df[['PATH', 'Label']].iloc[num:num+500]
    test_image_paths = test_set_df['PATH'].tolist()
    test_labels = test_set_df['Label'].tolist()

    # Define image directory
    image_dir = 'TB/images/'

    X = preprocess_images(sample_image_paths)
    X_test = preprocess_images(test_image_paths)


    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df['Label'])
    y_encoded = label_encoder.transform(sample_labels)
    y_test_encoded = label_encoder.transform(test_labels)

    # One-hot encode labels
    y_categorical = to_categorical(y_encoded)
    y_test_categorical = to_categorical(y_test_encoded)
    input_tensor = Input(shape=(224, 224, 3))

    base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    #Adjust the number of classes according to your dataset
    num_classes = len(label_encoder.classes_)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X, y_categorical, batch_size=16, epochs=10, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test, y_test_categorical)

    print("RESNET50:")
    print(f'Test Accuracy with {num} samples: {accuracy * 100:.2f}%')
    results.write("RESNET50,"+str(num)+","+str(accuracy * 100)+"\n")

    X = preprocess_images_svm(sample_image_paths)
    X_test = preprocess_images_svm(test_image_paths)
    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df['Label'])
    y_encoded = label_encoder.transform(sample_labels)
    y_test_encoded = label_encoder.transform(test_labels)
    
    # One-hot encode labels
    y_categorical = to_categorical(y_encoded)
    y_test_categorical = to_categorical(y_test_encoded)
    
    model_svm = SVM(degree=8)
    model_tisvm = TISVM(degree=8)
    model_locsvm = LOCSVM(degree=9, filter=9)
    model_lrisvm = LRISVM(degree=8, filter=3)
    model_ltisvm = LTISVM(degree=8,filter=7)
    model_risvm = RISVM(degree=8)
    model_riisvm = RIISVM(degree=8)
    model_tirisvm = TIRISVM(degree=8)
    model_ltirisvm = LTIRISVM(degree=8,filter=7)

    time1 = time.time()
    svclassifier, train_acc = model_svm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_svm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print("SVM (",time2-time2,"}s):")
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("SVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_tisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_tisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('TISVM (',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("TISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_locsvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_locsvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LOCSVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LOCSVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_lrisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_lrisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LRISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LRISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_ltisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_ltisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LTISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LTISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    #model_ltirisvm

    time1 = time.time()
    svclassifier, train_acc = model_ltirisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_ltirisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LTIRISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LTIRSVM,"+str(num)+","+str(test_acc * 100)+"\n")

# To run MICROGRAPH Dataset run next cell:

In [ ]:
import numpy as np
import time
from Model import SVM, TISVM, RISVM, LOCSVM, LTISVM, LRISVM, TIRISVM, LTIRISVM, RIISVM, KNN, TDSVM, ResNetModel
from DataLoader import load_data, train_valid_split
import ctypes
import torch
import pytorch_lightning as pl
import torch.utils.data as data_utils
from tqdm.autonotebook import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# Preprocess images
def preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        full_path = os.path.join(image_dir, img_path)
        img = load_img(full_path, target_size=(224, 224))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)
    
def preprocess_images_svm(image_paths):
    images = []
    for img_path in image_paths:
        full_path = os.path.join(image_dir, img_path)
        img = load_img(full_path, target_size=(28, 28),color_mode='grayscale')  # Resize for simplicity
        img_array = img_to_array(img)
        img_array = img_array.flatten() / 255.0  # Flatten and normalize
        images.append(img_array)
    return np.array(images)

# Load the CSV file
file_path = 'micrograph.csv'
df = pd.read_csv(file_path)

samples_num = [100, 200, 400]
results = open("result.txt","w",buffering=1)
results.write("model,number_of_samples,test_accuracy\n")

for num in samples_num:
    # Select the first 100 entries
    subset_df = df[['path', 'primary_microconstituent']].head(num)

    # Get the lists of image paths and labels from the sample
    sample_image_paths = subset_df['path'].tolist()
    sample_labels = subset_df['primary_microconstituent'].tolist()

    test_set_df = df[['path', 'primary_microconstituent']].iloc[num:num+500]
    test_image_paths = test_set_df['path'].tolist()
    test_labels = test_set_df['primary_microconstituent'].tolist()

    # Define image directory
    image_dir = 'images/'

    X = preprocess_images(sample_image_paths)
    X_test = preprocess_images(test_image_paths)


    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df['primary_microconstituent'])
    y_encoded = label_encoder.transform(sample_labels)
    y_test_encoded = label_encoder.transform(test_labels)

    # One-hot encode labels
    y_categorical = to_categorical(y_encoded)
    y_test_categorical = to_categorical(y_test_encoded)
    input_tensor = Input(shape=(224, 224, 3))

    base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    #Adjust the number of classes according to your dataset
    num_classes = len(label_encoder.classes_)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X, y_categorical, batch_size=16, epochs=10, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test, y_test_categorical)

    print("RESNET50:")
    print(f'Test Accuracy with {num} samples: {accuracy * 100:.2f}%')
    results.write("RESNET50,"+str(num)+","+str(accuracy * 100)+"\n")

    X = preprocess_images_svm(sample_image_paths)
    X_test = preprocess_images_svm(test_image_paths)
    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df['primary_microconstituent'])
    y_encoded = label_encoder.transform(sample_labels)
    y_test_encoded = label_encoder.transform(test_labels)
    
    # One-hot encode labels
    y_categorical = to_categorical(y_encoded)
    y_test_categorical = to_categorical(y_test_encoded)
    
    model_svm = SVM(degree=8)
    model_tisvm = TISVM(degree=8)
    model_locsvm = LOCSVM(degree=9, filter=9)
    model_lrisvm = LRISVM(degree=8, filter=3)
    model_ltisvm = LTISVM(degree=8,filter=7)
    model_risvm = RISVM(degree=8)
    model_riisvm = RIISVM(degree=8)
    model_tirisvm = TIRISVM(degree=8)
    model_ltirisvm = LTIRISVM(degree=8,filter=7)

    time1 = time.time()
    svclassifier, train_acc = model_svm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_svm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print("SVM (",time2-time2,"}s):")
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("SVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_tisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_tisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('TISVM (',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("TISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_locsvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_locsvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LOCSVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LOCSVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_lrisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_lrisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LRISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LRISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    time1 = time.time()
    svclassifier, train_acc = model_ltisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_ltisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LTISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LTISVM,"+str(num)+","+str(test_acc * 100)+"\n")

    #model_ltirisvm

    time1 = time.time()
    svclassifier, train_acc = model_ltirisvm.train(X, y_encoded)
    #eval_acc = model.evaluate(x_valid[0:val_list[idx], :], y_valid[0:val_list[idx]], svclassifier)
    test_acc = model_ltirisvm.evaluate(X_test, y_test_encoded, svclassifier)
    time2 = time.time()
    print('LTIRISVM ((',time2-time2,'}s):')
    print(f'Test Accuracy with {num} samples: {test_acc * 100:.2f}%')
    results.write("LTIRSVM,"+str(num)+","+str(test_acc * 100)+"\n")